# Set-Up

In [413]:
!pip install tetgen
!apt-get install -qq xvfb
!pip install pyvista panel -q
!pip install -q piglet pyvirtualdisplay
!pip install pygmsh
!pip install tqdm

In [2]:
# MUST RUN THIS CELL:
!pip uninstall -y h5py
!pip install h5py==2.9.0

     |████████████████████████████████| 287 kB 5.6 MB/s eta 0:00:01
  Created wheel for h5py: filename=h5py-2.9.0-cp39-cp39-linux_x86_64.whl size=4691332 sha256=77a3b93b82361775be6efe3bb933741e215c49a94060b5e887518a98c1878e6c
  Stored in directory: /root/.cache/pip/wheels/ef/54/5c/3fbdb9cfe071661699815cfd6b71ddf2d12d61d121a109a5e0
Successfully built h5py


In [43]:
# Fenics imports:
import dolfinx
import dolfinx.io
import dolfinx.plot
from dolfinx.cpp.mesh import CellType

# Numerics imports:
from petsc4py import PETSc
import numpy as np

# Mesh imports:
import gmsh
import pygmsh
import meshio
import ufl
from mpi4py import MPI
import tetgen

# Visualisation imports:
import pyvista

# Misc imports:
from math import sin, cos, pi, ceil, floor
import os

from tqdm import tqdm

# Mesh Conversion

In [44]:
def tetgen_to_meshio(tetgen_mesh):
    pyvista.save_meshio('temp.msh', tetgen_mesh)
    mesh = meshio.read('temp.msh')
    os.remove('temp.msh')
    return mesh

def meshio_to_grid(meshio_mesh):
    topology, cell_types = dolfinx.plot.create_vtk_topology(meshio_mesh, meshio_mesh.topology.dim)
    grid = pyvista.UnstructuredGrid(topology, cell_types, meshio_mesh.geometry.x)
    return grid

# Mesh Creation

In [45]:
def volume_from_surface(mesh, mindihedral):
    tet = tetgen.TetGen(mesh)
    tet.tetrahedralize(order=1, mindihedral=mindihedral) 
    tetgen_mesh = tet.grid
    return tetgen_mesh

In [46]:
def create_tetgen_volume(obj_dir, mindihedral):
    mesh = pyvista.read(obj_dir)
    tetgen_mesh = volume_from_surface(mesh, mindihedral)
    return tetgen_mesh

def create_meshio_volume(obj_dir, mindihedral):
    tetgen_mesh = create_tetgen_volume(obj_dir, mindihedral)
    meshio_mesh = tetgen_to_meshio(tetgen_mesh)
    meshio.write('breast.xdmf', meshio_mesh)
    with dolfinx.io.XDMFFile(MPI.COMM_WORLD, 'breast.xdmf', 'r') as f:
        meshio_mesh = f.read_mesh(name="Grid")
    for ext in ('.h5', '.xdmf'):
        os.remove('breast'+ext)
    return meshio_mesh

In [572]:
# Example:
obj_dir = 'breast_remeshed.obj'
mindihedral = 0.1
tetgen_mesh = create_tetgen_volume(obj_dir, mindihedral)
meshio_mesh = tetgen_to_meshio(tetgen_mesh)

In [551]:
from itertools import combinations
from math import pi
NUM_VERT=4
NUM_SUBDIV=4
EDGE_COMBOS = list(combinations(range(NUM_VERT),2))
FACE_COMBOS = list(combinations(range(NUM_VERT),3))
EPS = 1e-12
ORDERING_AXIS = 2


EL_0_ORDER = ((  (0,),   (0,1),   (0,1,2),   (0,2)),
               ((0,3), (0,1,3), (0,1,2,3), (0,2,3)))
EL_1_ORDER = ((  (1,),   (1,2),   (0,1,2),   (0,1)),
               ((1,3), (1,2,3), (0,1,2,3), (0,1,3)))
EL_2_ORDER = (( (2,),   (0,2),   (0,1,2),   (1,2)),
              ((2,3), (0,2,3), (0,1,2,3), (1,2,3)))
EL_3_ORDER = (((0,3), (0,1,3), (0,1,2,3), (0,2,3)),
              ( (3,),   (1,3),   (1,2,3),   (2,3)))
SUBDIV_ELS = (EL_0_ORDER, EL_1_ORDER, EL_2_ORDER, EL_3_ORDER)

SUBDIV_VERTS = (*[(i,) for i in range(NUM_VERT)],
                *EDGE_COMBOS,
                *FACE_COMBOS,
                tuple(i for i in range(NUM_VERT)))

SUBDIV_ORDER = []
for el in SUBDIV_ELS:
    face_idx = []
    for f in el:
        vert_idx = []
        for v_1 in f:
            vert_i = [idx for idx, v_2 in enumerate(SUBDIV_VERTS) if v_1==v_2]
            vert_idx.append(vert_i[0])
        face_idx.append(tuple(vert_idx))
    SUBDIV_ORDER.append(tuple(face_idx))
SUBDIV_ORDER = tuple(SUBDIV_ORDER) 

In [570]:
def compute_subdiv_coords(verts, coords):
    vert_coords = np.array([coords[v] for v in verts])
    
    # Order vertex along z axis:
    ordering = np.argsort(vert_coords[:, ORDERING_AXIS])
    vert_coords = vert_coords[ordering]
    
    edge_centres = [np.mean([vert_coords[idx] for idx in edge], axis=0) for edge in EDGE_COMBOS]
    face_centres = [np.mean([vert_coords[idx] for idx in faces], axis=0) for faces in FACE_COMBOS]
    vol_centre = np.mean(vert_coords, axis=0)
    subdiv_coords = np.vstack([*vert_coords, *edge_centres, *face_centres, vol_centre])
    return subdiv_coords

def find_exist_pts(hex_coords, coords):
    exist_pts = {}
    if coords.size>0:
        for i, c in enumerate(hex_coords):
            coord_dist = abs(coords - c)
            pts_same = np.all(coord_dist<EPS, axis=1)
            if pts_same.sum():
                exist_pts[i] = np.where(pts_same)[0].item()
    return exist_pts

def update_cells_and_coords(hex_coords, new_cells, new_coords):
    
    # First, see if any hex coordinates already exist within our mesh:
    exist_pts = find_exist_pts(hex_coords, new_coords)
    
    # Create new coordinates for those points not currently in the mesh:
    notexist_pts = {}
    num_coords = len(new_coords)
    for i, c in enumerate(hex_coords):
        if i not in exist_pts.keys():
            # Number this new point:
            notexist_pts[i] = num_coords
            # Add new coordinates to coordinates list:
            new_coords = np.vstack([new_coords, c])
            # Note we have one more point in mesh:
            num_coords += 1
    
    global_idx = exist_pts
    global_idx.update(notexist_pts)
    for elem in range(NUM_SUBDIV):
        cell_i = []
        for face in (0, 1):
            cell_i += [global_idx[i] for i in SUBDIV_ORDER[elem][face]]
        new_cells = np.vstack([new_cells, cell_i]) 
    return (new_cells, new_coords)

def order_cell_verts(el_idx, cell_i):
    for face in (0, 1):    
        ordering = SUBDIV_ORDER[el_idx][face]

def convert(mesh):
    old_coords = mesh.points
    new_cells, new_coords = np.empty((0,8), int), np.empty((0,3), float)
    for i, verts in enumerate(tqdm(mesh.cells[0].data)):
        # Compute coordinates of four hexahedra which will formed 
        # by dividing up this tetrahedron:
        subdiv_coords = compute_subdiv_coords(verts, old_coords)
        # Update the coordiantes list and cells list:
        new_cells, new_coords = update_cells_and_coords(subdiv_coords, new_cells, new_coords)
    
    new_mesh = meshio.Mesh(new_coords, [("hexahedron", new_cells)])
    return new_mesh

In [582]:
new_mesh = convert(meshio_mesh)
new_mesh.write('temp.msh')
new_mesh = pyvista.read('temp.msh')
visualise_mesh(meshio_mesh, subgrid=None, title=None)

100%|██████████| 507/507 [00:00<00:00, 689.64it/s]


VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, color_mappers=[LinearColorMapper(id='146...], height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

In [596]:
submesh = get_submesh(new_mesh, 0, 0.00)
visualise_mesh(new_mesh, subgrid=submesh, title=None)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

# Visualise Mesh

In [595]:
def visualise_mesh(grid, subgrid=None, title=None):
    pyvista.start_xvfb(wait=0.05)
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    title = "Breast Mesh" if title is None else title
    p.add_text(title, name="title", position="upper_edge")
    if subgrid is not None:
        p.add_mesh(grid, style="wireframe", color="k") 
        p.add_mesh(subgrid, color='white', lighting=True, show_edges=True) 
    else: 
        p.add_mesh(grid, show_edges=True, edge_color='k', color='white', lighting=False) # 
    
    p.show_axes()
    # p.show_bounds()
    p.show_grid()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

def get_submesh(mesh, axis, cutoff):
    cell_center = mesh.cell_centers().points
    mask = cell_center[:, axis] < cutoff
    cell_ind = mask.nonzero()[0]
    submesh = mesh.extract_cells(cell_ind)
    return submesh

# Mesh Visualisation

In [7]:
def get_submesh(tetgen_mesh, axis, cutoff):
    cell_center = tetgen_mesh.cell_centers().points
    mask = cell_center[:, axis] < cutoff
    cell_ind = mask.nonzero()[0]
    tetgen_submesh = tetgen_mesh.extract_cells(cell_ind)
    return tetgen_submesh

In [8]:
def visualise_tetgen_mesh(tetgen_mesh, tetgen_submesh=None, title=None, show_grid=False):
    pyvista.start_xvfb(wait=0.05)
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    title = "Breast Mesh" if title is None else title
    p.add_text(title, name="title", position="upper_edge")
    if tetgen_submesh is not None:
        p.add_mesh(tetgen_mesh, style="wireframe", color="k") 
        p.add_mesh(tetgen_submesh, lighting=True, show_edges=True) 
    else: 
        p.add_mesh(tetgen_mesh, show_edges=True, edge_color='k', lighting=False)
    p.show_axes()
    if show_grid:
        p.show_grid()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

In [57]:
def visualise_gmsh_mesh(gmsh_mesh, gmsh_submesh=None, title=None, show_grid=False):
    pyvista.start_xvfb(wait=0.05)
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    title = "Breast Mesh" if title is None else title
    p.add_text(title, name=title, position="upper_edge")
    if gmsh_submesh is not None:
        p.add_mesh(tetgen_mesh, style="wireframe", color="k") 
        p.add_mesh(tetgen_submesh, lighting=True, show_edges=True) 
    else: 
        p.add_mesh(gmsh_mesh, show_edges=True, edge_color='k', lighting=False)
    p.show_axes()
    if show_grid:
        p.show_grid()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

In [9]:
# Example:
tetgen_submesh = get_submesh(tetgen_mesh, axis=1, cutoff=80)
visualise_mesh(tetgen_mesh, tetgen_submesh, 
               title='Volume Mesh with Cross-Section',
               show_grid=True)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, axes={'xticker': {'ticks': arra...}, height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)

# Gravity Rotation

In [10]:
# Using Euler angles - see https://www.autonomousrobotslab.com/frame-rotations-and-representations.html
# Here, y_rot = theta, x_rot = psi
ANGLE_TO_RAD = pi/180
def rotate_gravity(g_vector, y_rot, x_rot):
    # NB: Negative associated with y so increasing y_rot goesin 'right direction'
    theta, psi = -ANGLE_TO_RAD*y_rot, ANGLE_TO_RAD*x_rot
    rot_matrix = np.array([[         cos(theta),        0,          -sin(theta)],
                           [sin(psi)*sin(theta),  cos(psi), sin(psi)*cos(theta)],
                           [cos(psi)*sin(theta), -sin(psi), cos(psi)*cos(theta)]])
    rotated_g = rot_matrix @ g_vector
    return rotated_g

In [11]:
# Test:
g_vector = np.array([1,0,0])
y_rot = 90
x_rot = 0
rotate_gravity(g_vector, y_rot, x_rot)

array([ 6.123234e-17,  0.000000e+00, -1.000000e+00])

# Apply Load to Breast

In [12]:
def apply_loading(obj_dir, y_rot, x_rot, E, nu, rho, g, elem_order, num_steps, mindihedral):
    
    mesh = create_meshio_volume(obj_dir, mindihedral)
    V = dolfinx.VectorFunctionSpace(mesh, ("CG", elem_order))
    
    # Create lambda and mu fields:
    lambda_ = E*nu/((1+nu)*(1-2*nu))
    mu = E/(2*(1+nu))
    
    # Apply fixed BC:
    fixed = lambda x: x[0] < 10
    fixed_facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, fixed)
    facet_tag = dolfinx.MeshTags(mesh, mesh.topology.dim-1, fixed_facets, 1)
    u_bc = dolfinx.Function(V)
    with u_bc.vector.localForm() as loc:
        loc.set(0)
    left_dofs = dolfinx.fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.indices[facet_tag.values==1])
    bcs = [dolfinx.DirichletBC(u_bc, left_dofs)]
    
    B = dolfinx.Constant(mesh, (0, 0, 0))
    T = dolfinx.Constant(mesh, (0, 0, 0))
    v = ufl.TestFunction(V)
    u = dolfinx.Function(V)

    d = len(u)
    I = ufl.variable(ufl.Identity(d))
    F = ufl.variable(I + ufl.grad(u))
    C = ufl.variable(F.T * F)
    Ic = ufl.variable(ufl.tr(C))
    J  = ufl.variable(ufl.det(F))
    psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lambda_ / 2) * (ufl.ln(J))**2
    P = ufl.diff(psi, F)
    
    metadata = {"quadrature_degree": elem_order}
    ds = ufl.Measure('ds', subdomain_data=facet_tag, metadata=metadata)
    dx = ufl.Measure("dx", metadata=metadata)
    F = ufl.inner(ufl.grad(v), P)*dx - ufl.inner(v, B)*dx - ufl.inner(v, T)*ds(2) 
    
    problem = dolfinx.fem.NonlinearProblem(F, u, bcs)
    solver = dolfinx.NewtonSolver(MPI.COMM_WORLD, problem)

    solver.atol = 1e-3
    solver.rtol = 1e-3
    solver.convergence_criterion = "incremental"
    
    g_vector = g*np.array([1,0,0])
    g_vector = rotate_gravity(g_vector, y_rot, x_rot)
    
    f_step = rho*g_vector/num_steps
    for n in range(num_steps):
        print(f"Performing load step {n+1}/{num_steps}")
        for i, f_i in enumerate(f_step):
            B.value[i] = (n+1)*f_i
        num_its, converged = solver.solve(u)
        assert(converged)
        u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    
    return (u, mesh)

# Visualise Breast Deformation

In [13]:
def plot_deformation(meshio_mesh, uh):
    mesh = meshio_mesh
    pyvista.start_xvfb(wait=0.05)
    topology, cell_types = dolfinx.plot.create_vtk_topology(mesh, mesh.topology.dim)
    grid = pyvista.UnstructuredGrid(topology, cell_types, mesh.geometry.x)
    
    p = pyvista.Plotter(notebook=True, window_size=[960,480]) #
    
    p.add_text("Deformed configuration", name="title", position="upper_edge")
    
    grid["u"] = uh.compute_point_values().real 
    actor_0 = p.add_mesh(grid, style="wireframe", color="k")
    warped = grid.warp_by_vector("u", factor=1.5)
    actor_1 = p.add_mesh(warped)
    
    p.show_axes()
    viewer = p.show(jupyter_backend='panel', return_viewer=True)
    return viewer

# Call Functions

In [14]:
# Fixed parameters:
elem_order = 2
W = 40 # in mm
L = 90 # in mm
nu = 0.33 # dimensionless
rho = 0.00102 # in g mm^-3
g = 9.81 # in m s^-2
num_steps = 5

# Variables:
y_rot = 90
x_rot = -90
E =  5 # in mPa

# Mesh information:
obj_dir = 'breast.obj'
mindihedral = 20

In [17]:
u, mesh = apply_loading(obj_dir, y_rot, x_rot, E, nu, rho, g, elem_order, num_steps, mindihedral)

Performing load step 1/5
Performing load step 2/5
Performing load step 3/5
Performing load step 4/5
Performing load step 5/5


In [18]:
plot_deformation(mesh, u)

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, color_mappers=[LinearColorMapper(id='101...], height=480, orientation_widget=True, sizing_mode='stretch_width', width=960)